In [ ]:
ORORG = "rapidsai"
REPO = "cudf"
BOTS = ["dependabot[bot]", "GPUtester", "github-actions[bot]"]
import os  # noqa: E402

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
repo: str = REPO

In [ ]:
    import pandas as pd
    from langchain_experimental.agents import create_pandas_dataframe_agent
    from langchain_openai import OpenAI as OpenAI_langchain
    from langchain_openai import OpenAIEmbeddings
    from pymilvus import MilvusClient

    df_issues = pd.read_parquet(f"{repo}_issue_summary.parquet")
    # drop the issue_text column as context is too large for agent
    # we will use the vector database instead
    df_issues = df_issues.drop(
        columns=[
            "issue_text",
            "LLM_title_subject",
            "label_names",
            "issue_text_tokens",
            "issue_created_at",
            "issue_updated_at",
            "issue_reactions.+1",
            "issue_reactions.-1",
            "issue_reactions.laugh",
            "issue_reactions.hooray",
            "issue_reactions.confused",
            "issue_reactions.heart",
            "issue_reactions.rocket",
            "issue_reactions.eyes",
            "issue_user.login_location_lat",
            "issue_user.login_location_lon",
        ]
    )
    # Create simple column names to help the agent
    df_issues = df_issues.rename(
        columns={
            "number": f"{repo}_issue_number",
            "title": f"{repo}_issue_title",
            "author_association": f"association_to_{repo}",
            "issue_reactions.total_count": "number_of_reactions_on_issue",
            "n_comments": "number_of_comments",
            "issue_user.login_email": "email",
            "issue_user.login_name": "name",
            "issue_user.login_company": "company",
            "issue_user.login_name_company": "name_company",
            "issue_user.login_location": "location",
            "issue_user.login_followers": "github_followers",
            "comment_reactions.total_count": "number_of_reactions_on_comments",
        }
    )
    print(df_issues["company"].value_counts())
    agent = create_pandas_dataframe_agent(
        OpenAI_langchain(
            temperature=0,
            model="gpt-3.5-turbo-instruct",
            openai_api_key=OPENAI_API_KEY,
        ),
        df_issues,
        verbose=True,
    )
    print(df_issues[df_issues["company"] == "Walmart"])
    response = agent_response(agent, "What issues are Walmart most interested in?")
    print(response)
    if ":" in response:
        response = response.split(":")[1].strip()

    question = f"What issues are similar to {response}?"

In [ ]:
response

In [ ]:
question = f"What issues are similar to {response}?"

In [ ]:
question

In [ ]:
    question_embeddings = embeddings_model.embed_documents([question])
    client = MilvusClient(f"./milvus_{repo}.db")
    res = client.search(
        collection_name="issue_text",
        data=question_embeddings,
        limit=2,
    )

In [ ]:
res = client.search(
    collection_name="issue_text",
    data=question_embeddings,
    limit=2,
)
similar_issue_1 = res[0][0]["id"]
similar_issue_2 = res[0][1]["id"]

In [ ]:
df_issues[df_issues[f"{repo}_issue_number"] == similar_issue_1]

In [ ]:
df_issues[df_issues[f"{repo}_issue_number"] == similar_issue_2]

In [ ]:
df